<a href="https://colab.research.google.com/github/pearl-yu/twitch_project/blob/main/raid_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/twitch_data

/content/drive/MyDrive/twitch_data


In [5]:
%ls

1013713136.mp4                  raid_VNCEOOfficial.html  sentiments3
chat_df.csv                     sentiments               sentiments4
chat_features_womeaningful.csv  sentiments1              sentiments5
data_vid_df_l.csv               sentiments2              video_df_merge.csv


In [103]:
# just want to take a look at the chats containing raid information
chat_df = pd.read_csv('chat_df.csv',lineterminator='\n',index_col=0) 
chat_df.head()

temp_table = chat_df[chat_df['messages'].str.contains('raid',na=False)]
temp_table

,video_url,timestamp,time_in_seconds,author_id,author_name,author_bio,messages
0,https://www.twitch.tv/videos/1619944979,1665353968134000,34.434,536714910,Mar0n1_,NaN,yo first
1,https://www.twitch.tv/videos/1619944979,1665353970195000,36.495,39378978,Azkii,"Hello, I am Azkii. Proud member of the Toad Sq...",Have good steam!
2,https://www.twitch.tv/videos/1619944979,1665353976374000,42.674,821199354,itzBaMofficial,I play Valorant at the moment,hi
3,https://www.twitch.tv/videos/1619944979,1665353978426000,44.726,519136869,youngmoneybandz,hey nerd,yoooo
4,https://www.twitch.tv/videos/1619944979,1665353978792000,45.092,477382692,shanenobi,NaN,!crosshair


In [127]:
# to get the list of streamers in sample, also will later merge raid data into this
# from independent_variable_chat_exploration.ipynb output
features_vid_df_lag = pd.read_csv('features_vid_df_lag.csv',lineterminator='\n',index_col=0) 
features_vid_df_lag.columns

Index(['user_id', 'video_id', 'video_url', 'num_messages', 'message_len',
       'video_id.1', 'num_messages_per_min', 'user_id.1', 'user_name',
       'duration_min', 'created_at', 'title', 'view_count', 'broadcaster_type',
       'broadcast_description', 'broadcaster_created_at',
       'avg_messages_per_min', 'sentiment_positive_percent', 'sentiment_score',
       'video_url_lag', 'num_messages_lag', 'message_len_lag', 'video_id_lag',
       'num_messages_per_min_lag', 'user_id_lag', 'user_name_lag',
       'duration_min_lag', 'created_at_lag', 'title_lag', 'view_count_lag',
       'broadcaster_type_lag', 'broadcast_description_lag',
       'broadcaster_created_at_lag', 'avg_messages_per_min_lag',
       'sentiment_positive_percent_lag', 'sentiment_score_lag',
       'time_after_last_stream', 'time_after_last_stream_end', 'user_id_post',
       'created_at_post', 'time_till_next_stream',
       'time_till_last_stream_afterend'],
      dtype='object')

In [128]:
print(features_vid_df_lag['video_url'].nunique())
print(features_vid_df_lag['user_id'].nunique())

91
12


In [129]:
# video_df.user_name.unique()
features_vid_df_lag.user_name.unique()

array(['zackrawrr', 'Methodz', 'SypherPK', 'iiTzTimmy', 'AkzOnTop',
       'VNCEOfficial', 'Rainbow6', 'moistcr1tikal', 'General_Mars',
       'Murda10k', 'PatProzyy', 'prxd4v41'], dtype=object)

## Parse raid data


In [17]:
import requests # to fetch the raw HTML from our page
import csv
from bs4 import BeautifulSoup # to extract just the information we are looking for

In [77]:
# reference 1: https://github.com/Vidito/webscraping_housing/blob/main/main.py
def get_data(soup_object):
    table = soup.find('table', class_="w-full rounded-lg bg-card")

    # scrapes the table head element
    outputHead = []
    for th in table.find_all('th'):  # find_all() function to find the elements we are looking for
        outputHead.append(th.text)  #write to a output list

    # traverses all the table rows
    outputRows = []
    for tr in table.find_all('tr'):
        tableColumns = tr.find_all('td')
        outputRow = []
        for tc in tableColumns:
            outputRow.append(tc.text.replace('\n', ''))
        outputRows.append(outputRow)

    return outputHead, outputRows

In [85]:
# testing the code with one streamer
#with open("raid_VNCEOfficial.html") as fp:
#    soup = BeautifulSoup(fp, 'html.parser')
#
#outputHead, outputRows = get_data(soup)
#raid_df= pd.DataFrame(outputRows,columns=outputHead)[1:].set_index(outputHead[0])
#
#raid_df.head()

In [243]:
raid_file_list = ['raid_VNCEOfficial.html', 'raid_PatProzyy.html', 'raid_prxd4v41.html', 'raid_Methodz.html',
        'raid_Rainbow6.html', 'raid_SypherPK.html', 'raid_iiTzTimmy.html',
       'raid_moistcr1tikal.html', 'raid_zackrawrr.html']

for i in range(len(raid_file_list)):
  with open('raid/' + raid_file_list[i] ) as fp:
    soup = BeautifulSoup(fp, 'html.parser')
    outputHead, outputRows = get_data(soup)
  print(i)
  if i == 0:
    raid_df= pd.DataFrame(outputRows,columns=outputHead)[1:].set_index(outputHead[0])
    print(raid_df.Whom.unique(),len(raid_df))
  else:
    temp = pd.DataFrame(outputRows,columns=outputHead)[1:].set_index(outputHead[0])
    raid_df = raid_df.append(temp)
    print(temp.Whom.unique(),len(temp))

0
[' VNCEOfficial'] 230
1
[' PatProzyy'] 28
2
[' prxd4v41'] 83
3
[' Methodz'] 84
4
[' Rainbow6'] 139
5
[' SypherPK'] 105
6
[' iiTzTimmy'] 408
7
[' moistcr1tikal'] 952
8
[' zackrawrr'] 171


In [244]:
# string to datetime format
raid_df['timestamp'] = pd.to_datetime(raid_df['Date'].str.strip(), format='%b %d, %Y at %H:%M')
raid_df['user_name'] = raid_df['Whom'].str.strip()
raid_df['num_raiders'] = raid_df['Number of viewers'].str.extract('(\d+)')

raid_df_clean = raid_df.drop(['Stream','Date','Whom','Number of viewers'], axis=1)

In [245]:
raid_df_clean.head()
print(raid_df_clean.shape)

(2200, 5)


In [246]:
raid_df_clean.head()

,Channel,Raided/Hosted,timestamp,user_name,num_raiders
,,,,,
1,SadlyNXT,raided,2022-11-03 18:59:00,VNCEOfficial,51
2,theholmanboy_games,raided,2022-11-03 18:57:00,VNCEOfficial,1
3,DHME5,raided,2022-11-03 18:42:00,VNCEOfficial,4
4,FezzSnezz,raided,2022-10-31 22:09:00,VNCEOfficial,2
5,nocaseace,raided,2022-10-28 19:54:00,VNCEOfficial,2


In [247]:
# This saves to google drive directly
raid_df_clean.to_csv('raid_df.csv')

## Towards identification

If incoming raids leads to exogenous change in viewership and chat intensity?

First let's take a look at the video level numbers. 
- Find the raids that happened during the video time range. 
- Then aggregate, Number of raids, total number of viewers raided. 

In [201]:
features_vid_df_lag['created_at'] = pd.to_datetime(features_vid_df_lag['created_at']).dt.tz_localize(None)
features_vid_df_lag['ended_at'] = features_vid_df_lag['created_at'] + pd.to_timedelta(features_vid_df_lag['duration_min'], unit='m') 

In [165]:
#features_vid_df_lag.user_name
#raid_df.Whom.unique()

In [248]:
#for i in range(2):
for i in range(len(features_vid_df_lag.user_name.unique())):
  streamer = features_vid_df_lag.user_name.unique()[i]
  temp_vid_table = features_vid_df_lag[features_vid_df_lag.user_name == streamer][['user_name','video_id','created_at','ended_at']]
  temp_raid_table = raid_df_clean[raid_df_clean['user_name'] == streamer]
  temp_raid_table = temp_raid_table.loc[(temp_raid_table['timestamp'] >= temp_vid_table['created_at'].min()) & (temp_raid_table['timestamp']<= temp_vid_table['created_at'].max())]
  print(temp_vid_table['user_name'].unique(),temp_raid_table.user_name.unique())
  
  temp = pd.merge(temp_vid_table, temp_raid_table,  how='left', left_on=['user_name'], right_on = ['user_name'])
  temp = temp[(temp['timestamp'] >= temp['created_at']) & (temp['timestamp'] <= temp['ended_at'])]

  if i ==0:
    vid_df_wraid = temp
    print(len(temp),len(vid_df_wraid))
  else: 
    vid_df_wraid = vid_df_wraid.append(temp)
    print(len(temp),len(vid_df_wraid))

vid_df_wraid = vid_df_wraid.reset_index(drop=True)

['zackrawrr'] ['zackrawrr']
4 4
['Methodz'] ['Methodz']
2 6
['SypherPK'] ['SypherPK']
0 6
['iiTzTimmy'] ['iiTzTimmy']
0 6
['AkzOnTop'] []
0 6
['VNCEOfficial'] ['VNCEOfficial']
2 8
['Rainbow6'] ['Rainbow6']
2 10
['moistcr1tikal'] ['moistcr1tikal']
11 21
['General_Mars'] []
0 21
['Murda10k'] []
0 21
['PatProzyy'] ['PatProzyy']
0 21
['prxd4v41'] ['prxd4v41']
12 33


In [249]:
vid_df_wraid.to_csv('vid_df_wraid.csv')

In [253]:
vid_df_wraid.head()

,user_name,video_id,created_at,ended_at,Channel,Raided/Hosted,timestamp,num_raiders
0,zackrawrr,1610656682,2022-10-06 05:28:07,2022-10-06 09:26:50.999999980,ArcAngL,raided,2022-10-06 09:01:00,4
1,zackrawrr,1610656682,2022-10-06 05:28:07,2022-10-06 09:26:50.999999980,MeatCuddle,raided,2022-10-06 08:00:00,5
2,zackrawrr,1610656682,2022-10-06 05:28:07,2022-10-06 09:26:50.999999980,EternalNess,raided,2022-10-06 07:32:00,1
3,zackrawrr,1610656682,2022-10-06 05:28:07,2022-10-06 09:26:50.999999980,emialbasta,raided,2022-10-06 07:29:00,4
4,Methodz,1605572910,2022-09-30 18:40:36,2022-09-30 22:17:30.000000000,Hitchariide,raided,2022-09-30 21:25:00,534


In [267]:
raid_sum_df = vid_df_wraid.groupby('video_id')['num_raiders'].sum().reset_index()
features_vid_df_lag_raid = pd.merge(features_vid_df_lag, raid_sum_df,  how='outer', left_on=['video_id'], right_on = ['video_id'])

In [279]:
features_vid_df_lag_raid['num_raiders'].fillna(0,inplace=True)
features_vid_df_lag_raid['num_raiders'] = features_vid_df_lag_raid['num_raiders'].astype(int)

In [280]:
raid_sum_df

,video_id,num_raiders
0,1591613035,221122
1,1592319276,910
2,1595352118,33
3,1598015150,7540
4,1605572910,534900
5,1606861627,5
6,1606964718,4115731679917
7,1607634101,14
8,1610256098,27
9,1610306079,1


In [269]:
# print(len(features_vid_df_lag),len(features_vid_df_lag_raid))

In [281]:
features_vid_df_lag_raid.isnull().sum()

user_id                            0
video_id                           0
video_url                          0
num_messages                       0
message_len                        0
video_id.1                         0
num_messages_per_min               0
user_id.1                          0
user_name                          0
duration_min                       0
created_at                         0
title                              0
view_count                         0
broadcaster_type                   2
broadcast_description              0
broadcaster_created_at             0
avg_messages_per_min               0
sentiment_positive_percent         0
sentiment_score                    0
video_url_lag                     12
num_messages_lag                  12
message_len_lag                   12
video_id_lag                      12
num_messages_per_min_lag          12
user_id_lag                       12
user_name_lag                     12
duration_min_lag                  12
c

In [282]:
features_vid_df_lag_raid.to_csv('features_vid_df_lag_raid.csv')

In [283]:
features_vid_df_lag_raid[['num_raiders','num_messages_per_min','num_messages','message_len','duration_min']].corr()

,num_raiders,num_messages_per_min,num_messages,message_len,duration_min
num_raiders,1.000000,0.446141,0.414717,-0.044104,0.067284
num_messages_per_min,0.446141,1.000000,0.801703,-0.232958,-0.022404
num_messages,0.414717,0.801703,1.000000,-0.129868,0.409100
message_len,-0.044104,-0.232958,-0.129868,1.000000,0.049782
duration_min,0.067284,-0.022404,0.409100,0.049782,1.000000


## Ignore code down below

In [117]:
raid_df['Date'][4]

'Oct 28, 2022 at 19:54'

In [ ]:
# makes a GET request to a URL and returns the text from that page.
def get_page(url):
    return requests.get(url).content

In [ ]:
page = get_page('https://streamscharts.com/tools/raid-finder/vnceofficial')
page

In [ ]:
soup = BeautifulSoup(page, 'html.parser') # parses the document using a HTML parser

In [34]:
lists = soup.find_all('th')

In [ ]:
lists

In [ ]:
import re

In [30]:
url = "raid_VNCEOOfficial.html"
page = open(url)

In [32]:
page.text()

AttributeError: ignored

In [19]:
soup = BeautifulSoup(page,'html.parser')

In [ ]:
print(table)

In [ ]:
header = table.find_all('th')

AttributeError: ignored

In [ ]:
print(table)

None


In [ ]:
soup